In [ ]:
import cv2
# import matplotlib
# import matplotlib.pyplot as plt
import numpy as np
# import glob
# import os.path as osp
import torch
import argparse
import os.path as osp
import sys
import cv2
from loguru import logger

import torch

import random
import os
import numpy as np

import cv2
# import matplotlib
# import matplotlib.pyplot as plt
import numpy as np
import glob
import torch
# import torch.nn as nn 
import time
import math

In [ ]:
path_gt = "/media/meysam/hdd/dataset/Dataset_UAV123/UAV123/anno/UAV123/car1_s.txt" 
img_files_path = glob.glob("/media/meysam/hdd/dataset/Dataset_UAV123/UAV123/data_seq/UAV123/car1_s/*")
img_files_path.sort()
frame = cv2.imread(img_files_path[0], cv2.IMREAD_COLOR)

In [ ]:
my_file = open(path_gt)

line = my_file.readline()
line = [int(l) for l in line[:-1].split(',')]
my_file.close()


In [ ]:
box = line


In [ ]:
box

In [ ]:
q_size = 200
target_sz_area = box[2]*box[3]
search_area_factor = 4.0
search_area = search_area_factor*search_area_factor*target_sz_area;
target_scale = np.sqrt(search_area)/q_size;

In [ ]:
output_sz = np.array([q_size, q_size])
sample_sz = target_scale * output_sz

In [ ]:
im = frame

In [1]:
pos = np.array([box[1] + box[3]/2.0, box[0] + box[2]/2.0])


NameError: name 'np' is not defined

In [ ]:
pos

In [ ]:
posl = pos.astype(np.int).copy()

In [ ]:
posl

In [ ]:
resize_factor = np.min(sample_sz.astype(np.float) / output_sz.astype(np.float)).item()
df = int(max(int(resize_factor - 0.1), 1))

In [ ]:
resize_factor

In [ ]:
sample_sz

In [ ]:
sz = sample_sz.astype(np.float) / df

In [ ]:
sz

In [ ]:
os = posl % df  # offset


In [ ]:
os

In [ ]:
posl = (posl - os) // df  # new position


In [ ]:
posl

In [ ]:
im2 = im[os[0].item()::df, os[1].item()::df, :]  # downsample

In [ ]:
im.shape[0]/df

In [ ]:
szl = np.maximum(np.round(sz), 2.0).astype(np.int)

In [ ]:
szl

In [ ]:
tl = posl - (szl - 1) // 2
br = posl + szl // 2 + 1

In [ ]:
tl*df

In [ ]:
br*df

In [ ]:
crop_xyxy = np.array([tl[1], tl[0], br[1], br[0]])
# warpAffine transform matrix
M_13 = crop_xyxy[0]
M_23 = crop_xyxy[1]
M_11 = (crop_xyxy[2] - M_13) / (output_sz[0] - 1)
M_22 = (crop_xyxy[3] - M_23) / (output_sz[1] - 1)
mat2x3 = np.array([
    M_11,
    0,
    M_13,
    0,
    M_22,
    M_23,
]).reshape(2, 3)

In [ ]:
im_patch = cv2.warpAffine(im2, mat2x3, 
                          (output_sz[0], output_sz[1]),
                          flags=(cv2.INTER_LINEAR | cv2.WARP_INVERSE_MAP),
                          borderMode=cv2.BORDER_CONSTANT,
                          borderValue=(0,0,0))

In [ ]:

def get_crop_numpy(im: np.ndarray, pos: np.ndarray, sample_sz: np.ndarray, output_sz: np.ndarray = None,
                   mode: str = 'constant', avg_chans=(0, 0, 0), max_scale_change=None):
    """Sample an image patch.

    args:
        im: Image
        pos: center position of crop
        sample_sz: size to crop
        output_sz: size to resize to
        mode: how to treat image borders: 'replicate' (default), 'inside' or 'inside_major'
        max_scale_change: maximum allowed scale change when using 'inside' and 'inside_major' mode
    """

    # if mode not in ['replicate', 'inside']:
    #     raise ValueError('Unknown border mode \'{}\'.'.format(mode))

    # copy and convert
    posl = pos.astype(np.int).copy()

    # Get new sample size if forced inside the image
    
    if mode == 'inside' or mode == 'inside_major':
        pad_mode = 'replicate'
        # im_sz = torch.tensor([im.shape[2], im.shape[3]], device=im.device)
        # shrink_factor = (sample_sz.float() / im_sz)
        im_sz = np.array([im.shape[0], im.shape[1]])
        shrink_factor = (sample_sz.astype(np.float) / im_sz)
        if mode == 'inside':
            shrink_factor = shrink_factor.max()
        elif mode == 'inside_major':
            shrink_factor = shrink_factor.min()
        shrink_factor.clamp_(min=1, max=max_scale_change)
        # sample_sz = (sample_sz.float() / shrink_factor).long()
        sample_sz = (sample_sz.astype(np.float) / shrink_factor).astype(np.int)

    # Compute pre-downsampling factor
    if output_sz is not None:
        # resize_factor = torch.min(sample_sz.float() / output_sz.float()).item()
        resize_factor = np.min(sample_sz.astype(np.float) / output_sz.astype(np.float)).item()
        df = int(max(int(resize_factor - 0.1), 1))
    else:
        df = int(1)

    # sz = sample_sz.float() / df  # new size
    sz = sample_sz.astype(np.float) / df

    # Do downsampling
    if df > 1:
        os = posl % df  # offset
        posl = (posl - os) // df  # new position
        im2 = im[os[0].item()::df, os[1].item()::df, :]  # downsample
        # print(im2.shape)
        # print(os)
        # print(df)
    else:
        im2 = im

    # compute size to crop
    # szl = torch.max(sz.round(), torch.tensor([2.0], dtype=sz.dtype, device=sz.device)).long()
    szl = np.maximum(np.round(sz), 2.0).astype(np.int)

    # Extract top and bottom coordinates
    tl = posl - (szl - 1) // 2
    br = posl + szl // 2 + 1

    # Shift the crop to inside
    if mode == 'inside' or mode == 'inside_major':
        # im2_sz = torch.LongTensor([im2.shape[2], im2.shape[3]])
        # shift = (-tl).clamp(0) - (br - im2_sz).clamp(0)
        im2_sz = np.array([im2.shape[0], im2.shape[1]], dtype=np.int)
        shift = np.clip(-tl, 0) - np.clip(br - im2_sz, 0)
        tl += shift
        br += shift

        # outside = ((-tl).clamp(0) + (br - im2_sz).clamp(0)) // 2
        # shift = (-tl - outside) * (outside > 0).long()
        outside = (np.clip(-tl, 0) - np.clip(br - im2_sz, 0)) // 2
        shift = (-tl - outside) * (outside > 0).astype(np.int)
        tl += shift
        br += shift

        # Get image patch
        # im_patch = im2[...,tl[0].item():br[0].item(),tl[1].item():br[1].item()]

    crop_xyxy = np.array([tl[1], tl[0], br[1], br[0]])
    # warpAffine transform matrix
    M_13 = crop_xyxy[0]
    M_23 = crop_xyxy[1]
    M_11 = (crop_xyxy[2] - M_13) / (output_sz[0] - 1)
    M_22 = (crop_xyxy[3] - M_23) / (output_sz[1] - 1)
    mat2x3 = np.array([
        M_11,
        0,
        M_13,
        0,
        M_22,
        M_23,
    ]).reshape(2, 3)

    # cv2.imshow("im2", im2)
    # print("im2 shape ", im2.shape)
    # print(os)

    im_patch = cv2.warpAffine(im2, mat2x3, 
                              (output_sz[0], output_sz[1]),
                              flags=(cv2.INTER_LINEAR | cv2.WARP_INVERSE_MAP),
                              borderMode=cv2.BORDER_CONSTANT,
                              borderValue=tuple(map(int, avg_chans)))
    

    # cv2.imshow("im_patch", im_patch)
    # cv2.waitKey(0)
    # Get image coordinates
    patch_coord = df * np.concatenate([tl, br]).reshape(1, 4)
    scale = output_sz / (np.array([br[1] - tl[1] + 1, br[0] - tl[0] + 1]) * df)
    return im_patch, patch_coord, scale
